In [2]:
# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

#to read the data from excel to pandas dataframes
import pandas as pd

C:\Users\maybaheti\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
#Read the data in two separate dataframes
software_Catalog = pd.read_excel("Data.xlsx",sheet_name="Software Catalog")
Tagged_Catalog = pd.read_excel("Data.xlsx",sheet_name="Tagged Catalog")

In [4]:
#Function to remove regex (https://www. or http://www. from the website column)
def remove_prefix_website(column):
    return(column.str.replace('http\S', '', case=False).str.replace('://|//|www.', '', case=False))

In [5]:
#Function to compare the two values and get a fuzzy score for it using Levenstine Distance
#This function require four parameters:
#1. type_to_be_matched: Product / Website
#2. Client_df: Client Data
#3. Tagged_df = Tagged Data
#4. threshold: how much fuzzy score is exceptable
#It returns a dataframe with 3 columns: Client data,Tagged data and each of the row's fuzzy score

def match_columns(type_to_be_matched,Client_df,Tagged_df,threshold=90):
    matched_list = []
    
    for row in Client_df.index:
        client_name = Client_df.get_value(row,"Software Catalog "+str(type_to_be_matched))
        for columns in Tagged_df.index:
            Tagged_name=Tagged_df.get_value(columns,"Tagged "+str(type_to_be_matched))
            matched_token=fuzz.partial_ratio(client_name,Tagged_name)
            if matched_token>= threshold:
                matched_list.append([client_name,Tagged_name,matched_token])
    return(pd.DataFrame(matched_list))

In [5]:
# Calling remove_prefix_website function which we created above
Tagged_Catalog['Tagged Website'] = remove_prefix_website(Tagged_Catalog['Tagged Website'])
software_Catalog['Software Catalog Website'] = remove_prefix_website(software_Catalog['Software Catalog Website'])

In [6]:
# Calling match_columns function which we created above to get the fuzzy score for the products
matched_product = match_columns(type_to_be_matched="Product",Client_df=software_Catalog,Tagged_df=Tagged_Catalog,threshold=88)
matched_product.columns = ['Software Catalog Product','Tagged Product','Product_Fuzzy_Score']

C:\Users\maybaheti\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  del sys.path[0]
C:\Users\maybaheti\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app


In [7]:
#Merge the matched products with the websites of Client's data and Tagged data
df1 = pd.merge(matched_product,software_Catalog,on="Software Catalog Product")
df2 = pd.merge(df1,Tagged_Catalog,on="Tagged Product")

In [8]:
#Calculate the fuzzy score for the website as well
matched_list = []
for i in range(len(df2)):
    matched_token = fuzz.partial_ratio(df2['Tagged Website'][i],df2['Software Catalog Website'][i])
    matched_list.append([df2['Software Catalog Website'][i],df2['Tagged Website'][i],matched_token])
df2['Website_Fuzzy_Score'] =pd.DataFrame(matched_list)[2]

In [10]:
#Select only the relevant columns for the output
Matched_Products= df2[["Software Catalog Product","Software Catalog Website","Tagged Product","Tagged Website"]]

In [11]:
#Getting rid of some duplicates records in the data and push the data to the csv file.
Matched_Products.sort_values("Tagged Product").groupby('Software Catalog Product').tail(1).to_csv("Matched_Products.csv",index = False)

#We were able to match 176 Products out of 284 Tagged Product Category.